In [11]:
"""
======================================
Getting started programming with pyAFQ
======================================

There are two ways to :doc:`use pyAFQ </howto/getting_started>`: through the
command line interface, and by writing Python code. This tutorial will walk you
through the basics of the latter, using pyAFQ's Python Application Programming
Interface (API).

"""
import os.path as op

import matplotlib.pyplot as plt
import nibabel as nib
import plotly
import pandas as pd

from AFQ.api.group import GroupAFQ
import AFQ.data.fetch as afd
import AFQ.viz.altair as ava

In [12]:

##########################################################################
# Example data
# ------------
# pyAFQ assumes that the data is organized in a BIDS compliant directory.
# To get users started with this tutorial, we will download some example
# data and organize it in a BIDS compliant way (for more details on how
# BIDS is used in pyAFQ, refer to :ref:`bids_tutorial`).
#
# The following call dowloads a dataset that contains a single subject's
# high angular resolution diffusion imaging (HARDI) data, collected at the
# Stanford Vista Lab
#
# .. note::
#   See https://purl.stanford.edu/ng782rw8378 for details on dataset.
#
# The data are downloaded and organized locally into a BIDS compliant
# anatomical data folder (``anat``) and a diffusion-weighted imaging data
# (``dwi``) folder, which are both placed in the user's home directory under::
#
#   ``~/AFQ_data/stanford_hardi/``
#
# The data is also placed in a derivatives directory, signifying that it has
# already undergone the required preprocessing necessary for pyAFQ to run.
#
# The clear_previous_afq is used to remove any previous runs of the afq object
# stored in the `~/AFQ_data/stanford_hardi/` BIDS directory. Set it to None if
# you want to use the results of previous runs.

afd.organize_stanford_data(clear_previous_afq="track")


tracking_params = dict(n_seeds=50000,
                       random_seeds=True,
                       rng_seed=2022,
                       trx=True,
                       num_chunks=100)

myafq = GroupAFQ(
    bids_path=op.join(afd.afq_home, 'stanford_hardi'),
    preproc_pipeline='vistasoft',
    tracking_params=tracking_params,
    viz_backend_spec='plotly_no_gif')


INFO:AFQ:fetching Stanford HARDI data
INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /Users/asagilmore/.dipy/stanford_hardi 
INFO:AFQ:Dataset is already in place. If you want to fetch it again please first remove the folder /Users/asagilmore/AFQ_data/stanford_hardi/derivatives
INFO:AFQ:No seed mask given, using FA (or first scalar if none are FA)thresholded to 0.2
INFO:AFQ:No stop mask given, using FA (or first scalar if none are FA)thresholded to 0.2


In [13]:
# .. note::
#    Running the code below triggers the full pipeline of operations
#    leading to the computation of the tract profiles. Therefore, it
#    takes a little while to run (about 40 minutes, typically).

myafq.export_up_to("streamlines")

# myafq.cmd_outputs(cmd="rm",dependent_on="track")

import time

start = time.time()
myafq.export('streamlines')
end = time.time()
print(f"Elapsed time: {end - start} seconds")

In [5]:
myafq.cmd_outputs(cmd="rm",dependent_on="track")

In [ ]:


##########################################################################
# Visualizing the bundles and calculating act profiles:
# -----------------------------------------------------
# The pyAFQ API provides several ways to visualize bundles and profiles.
#
# First, we will run a function that exports an html file that contains
# an interactive visualization of the bundles that are segmented.
#
# .. note::
#    By default we resample a 100 points within a bundle, however to reduce
#    processing time we will only resample 50 points.
#
# Once it is done running, it should pop a browser window open and let you
# interact with the bundles.
#
# .. note::
#    You can hide or show a bundle by clicking the legend, or select a
#    single bundle by double clicking the legend. The interactive
#    visualization will also all you to pan, zoom, and rotate.

bundle_html = myafq.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

##########################################################################
# We can also visualize the tract profiles in all of the bundles. These
# plots show both FA (left) and MD (right) layed out anatomically.
# To make this plot, it is required that you install with
# `pip install pyAFQ[plot]` so that you have the necessary dependencies.
#

fig_files = myafq.export("tract_profile_plots")["01"]

##########################################################################
# .. figure:: {{ fig_files[0] }}
#


##########################################################################
# We can even use altair to visualize the tract profiles in all
# of the bundles. We provide a more customizable interface for visualizing
# the tract profiles using altair.
# Again, to make this plot, it is required that you install with
# `pip install pyAFQ[plot]` so that you have the necessary dependencies.
#
profiles_df = myafq.combine_profiles()
altair_df = ava.combined_profiles_df_to_altair_df(profiles_df)
altair_chart = ava.altair_df_to_chart(altair_df)
altair_chart.display()


##########################################################################
# We can check the number of streamlines per bundle, to make sure
# every bundle is found with a reasonable amount of streamlines.

bundle_counts = pd.read_csv(myafq.export("sl_counts")["01"], index_col=[0])
for ind in bundle_counts.index:
    if ind == "Total Recognized":
        threshold = 1000
    else:
        threshold = 10
    if bundle_counts["n_streamlines"][ind] < threshold:
        raise ValueError((
            "Small number of streamlines found "
            f"for bundle(s):\n{bundle_counts}"))

AttributeError: module 'dipy.tracking.local_tracking' has no attribute 'ParaLocalTracking'